In [249]:
import torch
import torch.nn as nn
from collections import defaultdict
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import tqdm
import math
from collections import defaultdict

In [198]:
FILL_IN = "FILL_IN"

In [277]:
# Dictionaries, {idx -> ch} and {ch -> idx}
itos = defaultdict(int)
stoi = defaultdict(int)
# Number of characters used to predict the target character in the MLP Language Model 
block_size = 3
# Batch size used in MLP Language Model
batch_size = 32
# Embedding dimension, per character
d_model = 10
# Hidden dimension for RNN and also MLP Language Models 
d_h = 200

# START = START token
stoi['.'] = 0
itos[0] = '.'

# Loop over all names and create mappings itos and stoi mapping a unique character to a idx
FILL_IN

In [278]:
assert len(stoi) == len(itos)
vocab_size = len(stoi)
assert vocab_size == 27

In [279]:
stoi

defaultdict(int,
            {'.': 0,
             'e': 1,
             'm': 2,
             'a': 3,
             'o': 4,
             'l': 5,
             'i': 6,
             'v': 7,
             's': 8,
             'b': 9,
             'p': 10,
             'h': 11,
             'c': 12,
             'r': 13,
             't': 14,
             'y': 15,
             'n': 16,
             'g': 17,
             'z': 18,
             'f': 19,
             'd': 20,
             'u': 21,
             'k': 22,
             'w': 23,
             'q': 24,
             'x': 25,
             'j': 26})

## BiGram Language Model
- Implement the Bigram Language Model
- Get all the relevent counts, then get the train dataset Perplexity
- Use the class notes

In [281]:
# Using the formulas in class, loop over each name and get the parameters
c1 = defaultdict(int)
c2 = defaultdict(int)
for name in open('names.txt', 'r'):
    # Lowercase and remove any whitespace at the end
    name = name.lower().strip()
    # Pad with START = '.' and STOP = '.'
    name = FILL_IN
    # Transform to integer
    name = FILL_IN
    # Get the counts for Bigrams and Unigrams
    FILL_IN

In [282]:
# Get perplexity
sumneglogp = 0
T = 0
for name in open('names.txt', 'r'):
    # Get rid of white space and lowercase
    name = name.lower().strip()
    # Get the length of the word, without padding
    T += FILL_IN
    # Don't pad the STOP since we are not generating; pad with START
    name = FILL_IN
    # Transform to integrs
    name = FILL_IN
    # Get the loss -log(p(name)); use that the log of the product is the sum of the logs
    FILL_IN
# Print the Perplexity
print('Preplexity: ', torch.pow(torch.tensor(sumneglogp / T ), 2).item()) 

Preplexity:  13.895605087280273


In [289]:
# Generate a random word using this distributon
# When you generate STOP, terminate
name = '.'
while True:
    c = stoi[name[-1]]
    # Make the distribution from c to any other word other than START
    p = []
    for d in range(vocab_size):
        # Use the same indicies as the dictionary we set up
        # Populate p
        FILL_IN
    #print(p)
    assert len(p) == vocab_size
    # Sample randmly from the probability using torch.Categorical
    c = FILL_IN
    # Offset by 1 since we want indices [1, 2, ..., vocab_size]
    if c.item() == 0:
        break
    else:
        name += itos[c.item()]
print('Generated name: ' , name[1:])
            

Generated name:  moryamayashion


## MLP Language Model

- Implement the MLP language model from below
- Look at page 7, Equation (1)
- https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf

In [290]:
x_data = []
y_data = []
for name in open('names.txt', 'r'):
    name = name.lower().strip()
    # Pad with block_size START tokens and 1 STOP token
    name = ''.join(block_size * ['.']) + name + '.'
    # Loop through name and get the (x, y) pairs
    # Add (x, y) to x_data and y_data and make sure you transform to characters
    # Make sure x_data and y_data have integers, use stoi
    for i in range(len(name) - block_size):
        FILL_IN

In [291]:
class MLPLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # An embedding for each character; vocab_size of them
        self.e = FILL_IN
        # H; should take in block_size * d_model vector and output d_h
        self.H = FILL_IN
        # U; should take in d_h vector and output vocab_size
        self.U = FILL_IN
        # W; for the skip connection, should take in block_size * d_model and output vocab_size
        self.W = FILL_IN

    # x should be (batch_size, block_size)
    def forward(self, x):
        FILL_IN

In [300]:
x_data[0], y_data[0]

([0, 0, 0], 18)

In [293]:
# Define a dataloader with x_data and y_data with batch_size
dl = FILL_IN

In [294]:
for xb, yb in dl:
    assert xb.shape == (batch_size, 3)
    assert yb.shape == (batch_size,)
    break

In [295]:
# Define the MLP model and the Adam optimizer learning rate 0.001
model = MLPLanguageModel()
optimizer = FILL_IN

In [296]:
total_loss = 0
total_ct = 0
total_epochs = 20

for _ in range(total_epochs):
    for xb, yb in dl:
        # Zero the gradients
        FILL_IN

        # Get the logits
        logits = FILL_IN
                
        # Compute the loss
        loss = FILL_IN

        # Get the new gradient
        FILL_IN

        # Clip the gradients to max norm 0.1
        # Use clid_grad_norm from torch
        FILL_IN
        
        # Do a gradient update
        FILL_IN

        # Get the loss for the batch and get the number of batches
        total_loss += loss.item()
        total_ct += 1

        # Print the loss
        if total_ct and total_ct % 500 == 0:
            print(total_loss / total_ct)
            total_loss = 0
            total_ct = 0

2.5894072308540346
2.457281219482422
2.407582850694656
2.3947769327163697
2.386825907945633
2.34425501203537
2.3441826746463774
2.3230127596855166
2.314083107471466
2.3187712411880494
2.3163943738937376
2.299647463083267
2.3065791826248168
2.2824714307785032
2.264568898677826
2.2898624875545504
2.284603733062744
2.256791707277298
2.2530757277011872
2.259875492334366
2.2480121746063233
2.2454004590511323
2.237783147096634
2.2407697353363036
2.248863885164261
2.2322758870124817
2.2193622727394104
2.223179480791092
2.2356123917102813
2.216988515138626
2.22616584610939
2.2130448422431948
2.203174202203751
2.2143791213035584
2.2166521270275115
2.2154732391834258
2.1883920991420744
2.2045855729579924
2.2106930553913116
2.211230798959732
2.191891407251358
2.1892505435943606
2.196866712808609
2.181224030017853
2.1818389432430267
2.186741435050964
2.1679062938690183
2.1833346343040465
2.1822933118343353
2.1826359000205993
2.205969117641449
2.1794409832954407
2.1966153411865235
2.195118425369263

KeyboardInterrupt: 

In [297]:
with torch.no_grad():
    # Get perplexity
    sumneglogp = 0
    T = 0
    for name in open('names.txt', 'r'):
        name = name.lower().strip()
        T += len(name)
        # Pad with block_size START tokens
        name = FILL_IN
        x_data = []
        y_data = []
        # Gather all the terms over the loss
        # Notice that we compute -log p(...abc)
        # Which is -log p(a | ...) - log p(b | a..) - log p(c | ba.)
        FILL_IN
        # Gather the loss over the name, for each term
        # You need to get the softmax loss for each term
        # Can either use the CrossEntropyLoss or do this manually
        # Compute the loss
        logits = FILL_IN

        # Use reduction "sum" so you don't need to worry about N
        loss = FILL_IN

        # Change to log base 2
        loss *= FILL_IN

        sumneglogp += loss

    print('Preplexity: ', torch.pow(sumneglogp.clone().detach() / T , 2).item())    

Preplexity:  10.598803520202637


In [311]:
# Generate a random word using this distributon
# When you generate STOP, terminate
name = ''.join(block_size * ['.'])
while True:
    # Get the idx
    c = FILL_IN
    # Make the distribution from c to any other word other than START
    p = FILL_IN
    # Randomly sample from p a new character
    c = FILL_IN
    if c.item() == 0:
        break
    else:
        name += itos[c.item()]
print('Generated name: ' , name[block_size:])

Generated name:  manue


## RNN Language Model
- For each name, run an RNN character by character
- Use the recursion x = Tanh()(Wh @ h + Wx @ x + bh + bx) and y = Softmax()(Wy h + by)
- Do not use the RNN Cell from PyTorch, do this manually as hinted below

In [313]:
class RNNLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Each token has an embedding of size vocab_size
        self.e = FILL_IN
        # Wh used to map hidden to hidden
        self.Wh = FILL_IN
        self.Wx = FILL_IN
        self.Wy = FILL_IN

    def forward(self, x, h):
        # Run through to get the embedding for the token
        # The embedding per token is the feature vector x  we pass into the
        # Represent x as an embedding
        x = FILL_IN
        # Get the hidden state
        h = FILL_IN
        # Get the logits we use to predict y
        z = FILL_IN
        # Return the z predicting y for the timestep we are at and the next hidden state
        return z, h

In [314]:
model = RNNLanguageModel()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [1]:
total_loss = 0
total_ct = 0
total_epochs = 5

for _ in range(total_epochs):
    for name in open('names.txt', 'r'):
        name = name.lower().strip()
        # Add the start and end padding token
        name = '.' + name + '.'
        # name[:-1]
        x_data = FILL_IN
        # name[1:]
        y_data = FILL_IN
        logits = []
        # Set the hidden state to random
        h = FILL_IN
        # Zero the grad
        FILL_IN
        
        # Loop through each token and get the new h and then pass it forward
        # Accumulate all the logits
        for x in x_data:
            FILL_IN
            FILL_IN
        
        # Put all the logits into one tensor
        logits = FILL_IN
                
        # Compute the loss
        loss = FILL_IN

        # Get the new gradient
        FILL_IN

        # Clip the gradients at max norm 0.1
        FILL_IN
        
        # Do a gradient update
        FILL_IN

        # Get the loss for the batch and get the number of batches
        total_loss += loss.item()
        total_ct += 1

        if total_ct and total_ct % 100 == 0:
            print(total_loss / total_ct)
            total_loss = 0
            total_ct = 0
        

NameError: name 'FILL_IN' is not defined

In [312]:
with torch.no_grad():
    # Get perplexity
    sumneglogp = 0
    T = 0
    for name in open('names.txt', 'r'):
        name = name.lower().strip()
        T += len(name)
        name = '.' + name
        # Get the name from index 0 to -1 exclusive end
        x_data = FILL_IN
        # Get the y from index 1 to end inclusive end
        y_data = FILL_IN
        # logits per token prediction
        logits = []
        # Initialize the h vector to random
        h = FILL_IN
        # Loop over each chracter in the name and pass h and this into the RNN
        # Get the new logit
        for x in x_data:
            # Get the int for x
            x = FILL_IN
            # Get z and h
            z, h = FILL_IN
            # Append to logit
            FILL_IN

        # Get all the logits for each character
        logits = FILL_IN

        # Compute the loss across all characters
        loss = FILL_IN
                
        # Change to log base 2
        # log2(x) = ln(x) / ln(2)
        loss *= FILL_IN

        sumneglogp += FILL_IN
        
    # sumneglogp is -log(p('.' + name1)) -log(p('.' + name2)) -log(p('.' + name3)) ...  
    # Divide by the appropriate term to get the answer we want 
    print('Preplexity: ', torch.pow(sumneglogp.clone().detach() / T , 2).item())

TypeError: forward() takes 2 positional arguments but 3 were given

In [328]:
# Generate a random word using this distributon
# Intialize the word with 
name = '.'
# Initialize h to random
h = FILL_IN
while True:
    # Make c to an integer
    c = FILL_IN
    # Make the distribution from c to any other word other than START
    logits, h = FILL_IN
    # Get p; use Softmax
    p = FILL_IN
    # Sample from p
    c = FILL_IN
    # If we generate '.', stop
    if c.item() == 0:
        break
    else:
        name += itos[c.item()]
print('Generated name: ' , name[1:])

Generated name:  zacarlan
